# Sep 27, 2024: visualize marginals on brain
nii plots

In [1]:
import csv
import os
import sys
import numpy as np
import pandas as pd
import scipy as sp 
import dill as pickle 
from os.path import join as pjoin
from itertools import product
from tqdm import tqdm
from copy import deepcopy
from pathlib import Path
import subprocess
from scipy import sparse, stats
from multiprocessing import Pool
import glob
import random

import arviz as az

import ants
from nipype.interfaces import afni

from itertools import product, combinations
import multiprocessing as mp
from functools import partial

# networks
import graph_tool.all as gt

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.cm import rainbow

plt.rcParamsDefault['font.family'] = "sans-serif"
plt.rcParamsDefault['font.sans-serif'] = "Arial"
plt.rcParams['font.size'] = 14
plt.rcParams["errorbar.capsize"] = 0.5

import cmasher as cmr  # CITE ITS PAPER IN YOUR MANUSCRIPT
import colorcet as cc

# ignore user warnings
import warnings
warnings.filterwarnings("ignore") #, category=UserWarning)

In [2]:
class ARGS():
    pass

args = ARGS()

args.SEED = 100

def set_seed(args):
    gt.seed_rng(args.SEED)
    np.random.seed(args.SEED)

set_seed(args)

In [3]:
args.type = 'spatial'
args.roi_size = 225
args.maintain_symmetry = True
args.brain_div = 'whl'
args.num_rois = 162

PARC_DESC = (
    f'type-{args.type}'
    f'_size-{args.roi_size}'
    f'_symm-{args.maintain_symmetry}'
    f'_braindiv-{args.brain_div}'
    f'_nrois-{args.num_rois}'
)

In [4]:
args.GRAPH_DEF = f'constructed'
args.GRAPH_METHOD = f'pearson-corr'
args.THRESHOLDING = f'positive'
args.EDGE_DEF = f'binary'
args.EDGE_DENSITY = 25
args.LAYER_DEF = f'individual'
args.DATA_UNIT = f'ses'

BASE_path = f'{os.environ["HOME"]}/mouse_dataset'
PARCELS_path = f'{BASE_path}/parcels'
ROI_path = f'{BASE_path}/roi_results_v2/{PARC_DESC}'
TS_path = f'{ROI_path}/runwise_timeseries'
ROI_RESULTS_path = (
    f'{ROI_path}'
    f'/graph-{args.GRAPH_DEF}/method-{args.GRAPH_METHOD}'
    f'/threshold-{args.THRESHOLDING}/edge-{args.EDGE_DEF}/density-{args.EDGE_DENSITY}'
    f'/layer-{args.LAYER_DEF}/unit-{args.DATA_UNIT}'
)
GRAPH_path = f'{ROI_RESULTS_path}/graphs'
os.system(f'mkdir -p {GRAPH_path}')
SBM_path = f'{ROI_RESULTS_path}/model-fits'
os.system(f'mkdir -p {SBM_path}')
ESTIM_path = f'{ROI_RESULTS_path}/estimates'
os.system(f'mkdir -p {ESTIM_path}/individual')
os.system(f'mkdir -p {ESTIM_path}/group')

0

In [5]:
parcels_img = ants.image_read(f'{PARCELS_path}/{PARC_DESC}_desc-parcels.nii.gz')
parcels = parcels_img.numpy()
roi_labels = np.loadtxt(f'{PARCELS_path}/{PARC_DESC}_desc-labels.txt')

In [6]:
parcels_img

ANTsImage (LPI)
	 Pixel Type : float (float32)
	 Components : 1
	 Dimensions : (58, 79, 45)
	 Spacing    : (0.2, 0.2, 0.2)
	 Origin     : (18.1, 2.7, -7.8)
	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.]

In [7]:
def concatenate(in_files, out_file):
    try:
        os.remove(out_file)
    except:
        pass

    tcat = afni.TCat()
    tcat.inputs.in_files = in_files
    tcat.inputs.out_file = out_file
    tcat.inputs.rlt = ''
    tcat.cmdline 
    tcat.run()

    for file in in_files:
        try:
            os.remove(file)
        except:
            pass
    return None

def marginal_to_nifti(args, X, mode_id, folder, level=-1):
    args.num_rois, args.num_comms = X.shape

    in_files = []
    for idx_comm in range(args.num_comms):
        x = X[:, idx_comm]
        x_img = np.zeros_like(parcels)
        for idx, roi in enumerate(roi_labels):
            x_img += (parcels == roi) * (x[idx])
        
        file = f'{folder}/mode-{mode_id}_comm-{idx_comm}.nii.gz'
        parcels_img.new_image_like(x_img).to_filename(file)
        in_files.append(file)

    if level == -1:
        out_file = f'{folder}/mode-{mode_id:02d}.nii.gz'
    else:
        out_file = f'{folder}/level-{level}_mode-{mode_id:02d}.nii.gz'
    concatenate(in_files, out_file)
    return None

In [8]:
sbms_list = [
    (True, 'a'),
    (True, 'd'),
    (True, 'h'),
    (False, 'd'),
    (False, 'h'),
]

In [9]:
for sbm in sbms_list:
    args.dc, args.sbm = sbm

    args.nested = args.sbm == 'h'

    args.force_niter = 40000
    args.num_draws = int((1/2) * args.force_niter)

    def sbm_name(args):
        dc = f'dc' if args.dc else f'nd'
        dc = f'' if args.sbm in ['a'] else dc
        file = f'sbm-{dc}-{args.sbm}'
        return file

    SBM = sbm_name(args)
    print(SBM)

    indiv_files = sorted(glob.glob(f'{ESTIM_path}/individual/sub-*/partition-modes-group-aligned/{SBM}/desc-marginals-df.pkl', recursive=True))
    display(indiv_files)

    for indiv_file in tqdm(indiv_files):
        with open(indiv_file, 'rb') as f:
            indiv_marginals_df = pickle.load(f)

        out_folder = '/'.join(indiv_file.split('/')[:-1] + ['marginal-visuals'])
        os.system(f'mkdir -p {out_folder}')

        for idx, row in indiv_marginals_df.iterrows():
            if args.sbm in ['a', 'd', 'o']:
                marginal_to_nifti(args, row['pi'], row['mode_id'], out_folder)
            if args.sbm in ['h']:
                marginal_to_nifti(args, row['pi'], row['mode_id'], out_folder, level=row['level'])
        # break

sbm--a


['/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC01/partition-modes-group-aligned/sbm--a/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC02/partition-modes-group-aligned/sbm--a/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/graph-constructed/method-pearson-corr/threshold-positive/edge-binary/density-25/layer-individual/unit-ses/estimates/individual/sub-SLC03/partition-modes-group-aligned/sbm--a/desc-marginals-df.pkl',
 '/home/govindas/mouse_dataset/roi_results_v2/type-spatial_size-225_symm-True_braindiv-whl_nrois-162/gr

  0%|          | 0/10 [00:00<?, ?it/s]

240928-17:41:17,220 nipype.interface INFO:
	 stderr 2024-09-28T17:41:17.219935:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
240928-17:41:17,223 nipype.interface INFO:
	 stderr 2024-09-28T17:41:17.223135:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
240928-17:41:17,224 nipype.interface INFO:
	 stderr 2024-09-28T17:41:17.224093:*+ WARNING: Set TR of output dataset to 1.0 s
240928-17:41:17,261 nipype.interface INFO:
	 stderr 2024-09-28T17:41:17.261701:++ elapsed time = 0.0 s
240928-17:41:17,855 nipype.interface INFO:
	 stderr 2024-09-28T17:41:17.855705:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
240928-17:41:17,859 nipype.interface INFO:
	 stderr 2024-09-28T17:41:17.859798:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
240928-17:41:17,862 nipype.interface INFO:
	 stderr 2024-09-28T17:41:17.862317:*+ WARNING: Set TR of output dataset to 1.0 s
240928-17:41:17,919 nipype.interface INFO:
	 stderr 2024-09

 10%|█         | 1/10 [00:09<01:27,  9.75s/it]

...

 80%|████████  | 8/10 [05:41<01:25, 42.79s/it]

240928-18:01:01,809 nipype.interface INFO:
	 stderr 2024-09-28T18:01:01.809057:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
240928-18:01:01,826 nipype.interface INFO:
	 stderr 2024-09-28T18:01:01.826018:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
240928-18:01:01,828 nipype.interface INFO:
	 stderr 2024-09-28T18:01:01.828208:*+ WARNING: Set TR of output dataset to 1.0 s
240928-18:01:01,995 nipype.interface INFO:
	 stderr 2024-09-28T18:01:01.995406:++ elapsed time = 0.2 s
240928-18:01:03,592 nipype.interface INFO:
	 stderr 2024-09-28T18:01:03.592265:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
240928-18:01:03,605 nipype.interface INFO:
	 stderr 2024-09-28T18:01:03.605627:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
240928-18:01:03,608 nipype.interface INFO:
	 stderr 2024-09-28T18:01:03.608206:*+ WARNING: Set TR of output dataset to 1.0 s
240928-18:01:03,761 nipype.interface INFO:
	 stderr 2024-09

 90%|█████████ | 9/10 [06:28<00:44, 44.04s/it]

240928-18:01:48,157 nipype.interface INFO:
	 stderr 2024-09-28T18:01:48.157732:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
240928-18:01:48,162 nipype.interface INFO:
	 stderr 2024-09-28T18:01:48.162866:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
240928-18:01:48,164 nipype.interface INFO:
	 stderr 2024-09-28T18:01:48.163978:*+ WARNING: Set TR of output dataset to 1.0 s
240928-18:01:48,290 nipype.interface INFO:
	 stderr 2024-09-28T18:01:48.290067:++ elapsed time = 0.1 s
240928-18:01:49,376 nipype.interface INFO:
	 stderr 2024-09-28T18:01:49.376170:++ 3dTcat: AFNI version=AFNI_20.2.18 (Sep 17 2020) [64-bit]
240928-18:01:49,385 nipype.interface INFO:
	 stderr 2024-09-28T18:01:49.385251:*+ WARNING: can't apply -rlt option -- Not enough points per input dataset.
240928-18:01:49,387 nipype.interface INFO:
	 stderr 2024-09-28T18:01:49.387535:*+ WARNING: Set TR of output dataset to 1.0 s
240928-18:01:49,502 nipype.interface INFO:
	 stderr 2024-09

100%|██████████| 10/10 [07:10<00:00, 43.01s/it]
